In [3]:
import numpy as np
import matplotlib.pyplot as plt

# System parameters
Nx, Ny = 3, 3  # 3x3 lattice
N = Nx * Ny
J1, J2 = 1, 0.2  # coupling constants

# Pauli matrices
Sx = 0.5 * np.array([[0, 1], [1, 0]])
Sy = 0.5 * np.array([[0, -1j], [1j, 0]])
Sz = 0.5 * np.array([[1, 0], [0, -1]])
I = np.eye(2)

def build_interaction(i, j, op):
    """Construct the interaction term between sites i and j"""
    ops = [I] * N  # Initialize as a list of identity matrices
    ops[i] = ops[j] = op  # Set the i-th and j-th positions to the operator op
    term = ops[0]
    for op in ops[1:]:
        term = np.kron(term, op)  # Compute the Kronecker product
    return term

# Initialize the Hamiltonian
H = np.zeros((2**N, 2**N), dtype=complex)

# J1 term (nearest neighbor interaction)
nn_pairs = []
for x in range(Nx):
    for y in range(Ny):
        if x < Nx - 1:
            nn_pairs.append((x + y * Nx, (x + 1) + y * Nx))
        if y < Ny - 1:
            nn_pairs.append((x + y * Nx, x + (y + 1) * Nx))

for i, j in nn_pairs:
    for S in [Sx, Sy, Sz]:
        H += J1 * build_interaction(i, j, S)

# J2 term (next-nearest neighbor interaction)
nnn_pairs = []
for x in range(Nx):
    for y in range(Ny):
        if x < Nx - 1 and y < Ny - 1:
            nnn_pairs.append((x + y * Nx, (x + 1) + (y + 1) * Nx))
        if x > 0 and y < Ny - 1:
            nnn_pairs.append((x + y * Nx, (x - 1) + (y + 1) * Nx))

for i, j in nnn_pairs:
    for S in [Sx, Sy, Sz]:
        H += J2 * build_interaction(i, j, S)

# Solve for the ground state
E, V = np.linalg.eigh(H)
ground_state = V[:, 0]
print(f"Ground state energy: {E[0]/N:.6f}")

Ground state energy: -0.490552


In [5]:
import netket as nk
import numpy as np

L = 4  # 晶格长度
N = L * L  # 晶格中的格点数
J2 = 0.505  # 下一个最近邻相互作用的耦合常数

# 创建2D晶格
lattice = nk.graph.Grid(extent=[L, L], pbc=[True, True], max_neighbor_order=2)

# 创建Hilbert空间
hilbert = nk.hilbert.Spin(s=1/2, N=lattice.n_nodes)

# 创建Heisenberg自旋哈密顿量
hamiltonian = nk.operator.Heisenberg(hilbert=hilbert, graph=lattice, J=[1.0, J2])

# 使用Lanczos算法计算哈密顿量的本征值和本征向量
# evals, evecs = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)
evals = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=False)

# 输出基态能量
print('Exact ground-state energy E0 = ', evals[0] / (4*N))  # 基态能量除以四是因为netket里用的泡利矩，缺少1/2常数，两个相互作用就是1/4

Exact ground-state energy E0 =  -0.5278763469892256
